Code for Simulator - updates Geojson file with Completion status and time

In [63]:
import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxr
import random
from datetime import datetime, timedelta, timezone
import os
import geopandas as gpd
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
from rasterio.enums import Resampling
from tatc.schemas import TwoLineElements
from tatc.schemas import Point
from tatc.analysis import collect_observations
from tatc import utils
from tatc.schemas import Instrument
from tatc.schemas import WalkerConstellation, SunSynchronousOrbit
from tatc.utils import (
    swath_width_to_field_of_regard,
    along_track_distance_to_access_time,
)
import datetime
from tatc.analysis import collect_multi_observations
from tatc.schemas import Satellite
from datetime import datetime, timedelta, timezone
from tatc.schemas import Point
import logging
logging.basicConfig(level=logging.INFO)
from tatc.analysis import collect_ground_track
from tatc.analysis import compute_ground_track
from tatc.schemas import PointedInstrument, WalkerConstellation, SunSynchronousOrbit
from tatc.utils import swath_width_to_field_of_regard, swath_width_to_field_of_view
import pytz

class Simulator():
    def __init__(self):
        pass

    # This function uses the tatc library to define the  Snow_globe Constellation, we define tle, instrument 
    
    def const(self):
        roll_angle = (30 + 33.5)/2
        roll_range = (33.5 - 30)
        start = datetime(2018, 1, 1, tzinfo=timezone.utc)
        self.constellation = WalkerConstellation(
            name="SnowGlobe Ku",
            orbit=SunSynchronousOrbit(
                altitude=555e3, 
                equator_crossing_time="06:00:30", 
                equator_crossing_ascending=False,
                epoch=start
            ),
            number_planes=1,
            number_satellites=5,
            instruments=[
                PointedInstrument(
                    name="SnowGlobe Ku-SAR",
                    roll_angle=-roll_angle,
                    field_of_regard=2*roll_angle + swath_width_to_field_of_regard(555e3, 50e3),
                    along_track_field_of_view=swath_width_to_field_of_view(555e3, 50e3, 0),
                    cross_track_field_of_view=roll_range + swath_width_to_field_of_view(555e3, 50e3, roll_angle),
                    is_rectangular=True
                )
            ]
        )
        satellites = self.constellation.generate_members()
        self.satellite_dict = {sat.name: sat for sat in satellites}

    # This function reads the master geojson file and filter unprocessed requests
    # (non empty simulation status indicates, that location/request was processed) 
    # key output = self.filtered_req (pandas dataframe with filtered rows )  
    
    def user_request(self):
        self.req = gpd.read_file('Master_file')      
        self.filtered_req = self.req[self.req['simulation_status'].isna() | (self.req['simulation_status'] == "None")]         

    # This function uses tatc library multi - observations to generate the list of observation times
    # It uses all the locations from the user_request function, the simulation time at each step is given as input    
    # key output = self.combined_results, is a pandas data frame, column epoch returns the obs time (we sort it ascending)

    def opportunity(self):
        self.const()
        self.user_request()
        end = self._time + timedelta(2)
        combined_results = pd.DataFrame()
        for index,row in self.filtered_req.iterrows():    
            loc = Point(id=row['id'],latitude=row['latitude'],longitude=row['longitude'])
            results = collect_multi_observations(loc, self.constellation, self._time, end)
            combined_results = pd.concat([combined_results, results], ignore_index=True)    
        self.combined_results = combined_results.sort_values(by='epoch', ascending=True)
        
   # whenever the planner uploades geojson file we want requests to be updated
   # (to incorporate changes from optimizer or updates from simulation , ie simulation status updates)
   
    def execute(self,init_time, duration, time_step): 
        # intilization
        self.const()
        self.user_request()
        self._time = self._next_time = self._init_time = init_time
        self._duration = duration
        self._time_step = time_step

        while self._time < self._init_time + self._duration:
            counter = 0
            print(f"Current time {self._time}")            
            self._next_time = self._time + self._time_step
            print(f"advancing to {self._next_time}") 

            # updating user requests - read filtered rows
            # Logic if appender completes execution then execute below line
            
            self.user_request() # reading updated master file        

            # Error handler
            if self.filtered_req.empty:
                logging.info("No observations available. Skipping to next time step.")
                # Can use this condition to reset the master file
                # self._time = self._next_time
                continue
            
            self.opportunity() # updating observations list
            if self.combined_results.empty:
                logging.error("combined_results is empty! No observations until next time step! Skipping to next")
                # self._time = self._next_time
                continue
            

            self.rs = self.combined_results # writing the observations pandas dataframe to new variable
            self.observation_time = self.rs['epoch'].iloc[0] # latest possible observation            
            self.id = self.rs['point_id'].iloc[0] # point id for the above observation
            self.coord = self.rs['geometry'].iloc[0] # location for the observation
            self.sat = self.rs['satellite'].iloc[0] # satellite collecting the observation
            prev_observation_time = None
            # This below loop is written to handle the time step(1 day), there can be multiple observations within a day 
            # it loops through all the observations possible until the next time step

            while self.observation_time < self._next_time:
                if self.observation_time == prev_observation_time:
                    logging.warning("No progress in observations, breaking loop.")
                    break
                prev_observation_time = self.observation_time
                print(f"Next observation {self.observation_time}") 
                req = self.req # reads the requests file
                #req = pd.read_excel('requests.xlsx')
                req['completion_date'] = pd.to_datetime(req['completion_date'], errors='coerce')  # Ensure completion_date is datetime
                req['simulation_status'] = req['simulation_status'].astype(str)  # Ensure simulation_status is string
                # req['request_status'] = req['request_status'].astype(str)
                req['satellite'] = req['satellite'].astype(str)
                # format time as required in gejson file
                t = self.observation_time
                # t = self.observation_time.replace(tzinfo=None)
                
                # Groundtrack information

                # The below lines updates the excel file for the specific id corresponding to the latest observation time
                # ps note, the locations are not processed sequentially as in excel, id 13 can be processed before id 1 based on satellite location                            

                req.loc[req.id == self.id, 'completion_date'] = t
                req.loc[req.id == self.id, 'simulation_status'] = 'Completed'
                # req.loc[req.id == self.id, 'request_status'] = 'Completed'
                req.loc[req.id == self.id, 'satellite'] = self.sat

                # Groundtrack information
                sat_object = self.satellite_dict.get(self.sat)
                results = collect_ground_track(sat_object,[t],crs='spice')
                req.loc[req.id == self.id, 'geometry'] = results['geometry'].iloc[0]         
                
                # Save the updated DataFrame back to the Master Geojson file
                counter += 1
                req.to_file("Master_file", driver="GeoJSON")
                # Regenerating observations with respect to updated list
                # calling user_request and opprtunity will now exclude the entries processed above(sompleted status) and generate new list
                self.user_request()

                # Error handler
                if self.filtered_req.empty:
                    logging.info("No observations available. Skipping to next time step.")
                # Can use this condition to reset the master file
                    # self._time = self._next_time
                    continue
                
                self.opportunity() 

                if self.combined_results.empty:
                    logging.error("combined_results is empty! No observations until next time step! Skipping to next")
                    # self._time = self._next_time
                    continue

                self.rs = self.combined_results
                self.observation_time = self.rs['epoch'].iloc[0]
                self.id = self.rs['point_id'].iloc[0]
                self.coord = self.rs['geometry'].iloc[0]
                self.sat = self.rs['satellite'].iloc[0]
            # simulation advances to next time
            # filter data and write geojson
            if counter>0:
                self.user_request()
                # Filter data for each day(self.time)
                file_name = f"Simulator_Output_{pd.to_datetime(self._time).strftime('%Y-%m-%d')}"
                filtered_data = self.req[self.req['completion_date'].dt.date == pd.to_datetime(self._time).date()]               
                filtered_data.to_file(file_name, driver="GeoJSON")                

            self._time = self._next_time     


Executing the simulation

In [64]:
s = Simulator()
from datetime import datetime, timedelta, timezone

start = datetime(2019, 3, 10, tzinfo=timezone.utc)
duration = timedelta(days=5)
time_step = timedelta(days=1)
s.execute(start, duration, time_step)

Current time 2019-03-10 00:00:00+00:00
advancing to 2019-03-11 00:00:00+00:00


C:\Users\dramach6\AppData\Local\Temp\ipykernel_25188\125898820.py:158: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2019-03-10 00:02:56.166147500+00:00' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  req.loc[req.id == self.id, 'completion_date'] = t
INFO:pyogrio._io:Created 51 records


Next observation 2019-03-10 00:02:56.166147500+00:00


INFO:pyogrio._io:Created 51 records


Next observation 2019-03-10 00:03:10.059897+00:00


...

INFO:pyogrio._io:Created 51 records


Next observation 2019-03-10 12:17:53.440721500+00:00


INFO:pyogrio._io:Created 51 records


Next observation 2019-03-10 12:18:00.421009500+00:00
Next observation 2019-03-10 12:18:07.401137+00:00


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'Master_file'

In [66]:
master = gpd.read_file('Master_file')
filtered_req = master[master['simulation_status'].isna()] 
filtered_req
master.head(3)

,id,time,final_eta,Planner_geometry,centroid,latitude,longitude,expiration_date,simulation_status,completion_date,satellite,geometry
0,1,2019-03-10,0.003591,"POLYGON ((-91.94683 37.024602, -91.94683 37.63...",POINT (-92.252265 37.329427),37.329427,-92.252265,None,Completed,None,None,None


In [58]:
o_p = gpd.read_file('Simulator_Output_2019-03-11')
o_p

,geometry
